In [1]:
import torch
torch.cuda.mem_get_info()[0] / 1024**3

26.88134765625

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('wiki_dataset/rus_wikitext_10k_caption_reference_description_train.csv', index_col=0)

In [4]:
df.head()

,language,page_url,image_url,page_title,section_title,hierarchical_section_title,caption_reference_description,caption_attribution_description,caption_alt_text_description,mime_type,original_height,original_width,is_main_image,attribution_passes_lang_id,page_changed_recently,context_page_description,context_section_description
Unnamed: 0,,,,,,,,,,,,,,,,,
0,ru,https://ru.wikipedia.org/wiki/%D0%A3%D0%BC%D0%...,https://upload.wikimedia.org/wikipedia/commons...,Умершие в ноябре 2013 года,16 ноября,Умершие в ноябре 2013 года / 16 ноября,Владислав Захаревич,Русский: Ректор ЮФУ В.Г. ЗахаревичEnglish: Pro...,NaN,image/jpeg,1533,1677,False,True,True,"Это список известных людей, соответствующих ус...","Берберян, Арсен (75) — архиепископ Армянской а..."
5,ru,https://ru.wikipedia.org/wiki/%D0%91%D1%83%D1%...,https://upload.wikimedia.org/wikipedia/commons...,Буш (курган),NaN,Буш (курган),Зарисовка крупнейшего из Золотых ромбов курган...,English: design of the the Bush Barrow Lozenge...,NaN,image/jpeg,588,625,False,False,True,"Курган Буш, англ. Bush Barrow — памятник ранне...","Курган Буш, англ. Bush Barrow — памятник ранне..."
8,ru,https://ru.wikipedia.org/wiki/%D0%A2%D1%80%D0%...,https://upload.wikimedia.org/wikipedia/commons...,Трассовый моделизм,NaN,Трассовый моделизм,Трассовый моделизм. Прохождение моделями виража,日本語: YSR race Mr.ky,NaN,image/jpeg,540,960,False,False,False,Трассовый моделизм — вид автомодельного спорта...,Трассовый моделизм (также трассовые гонки или ...
15,ru,https://ru.wikipedia.org/wiki/%D0%9B%D0%B8%D0%...,https://upload.wikimedia.org/wikipedia/commons...,Лилейная трещалка,NaN,Лилейная трещалка,Спаривание,"Liloceris lilii (Scopoli, 1763), copula Englis...",NaN,image/jpeg,2336,3504,False,False,True,Треща́лка лиле́йная — насекомое семейства жуко...,Треща́лка лиле́йная (лат. Lilioceris lilii) — ...
16,ru,https://ru.wikipedia.org/wiki/%D0%91%D1%83%D1%...,https://upload.wikimedia.org/wikipedia/commons...,"Бурылин, Дмитрий Геннадьевич",Коллекционная деятельность,"Бурылин, Дмитрий Геннадьевич / Коллекционная д...",Музей Дмитрия Бурылина.,English: Museum of Industry and Art (Burylin M...,NaN,image/jpeg,3759,5012,False,True,False,Дми́трий Генна́дьевич Буры́лин — иваново-возне...,Всю свою жизнь Дмитрий Бурылин собирал коллекц...


In [5]:
df = df.drop(columns=['page_changed_recently', 'page_changed_recently', 'language', 'page_url', 'image_url', 'mime_type'])

In [6]:
columns = ['page_title', 'section_title', 'hierarchical_section_title', 'caption_reference_description', 
           'caption_attribution_description', 'context_page_description', 'context_section_description']

In [7]:
from tqdm import tqdm
df.loc[df.caption_reference_description.isna(), 'caption_alt_text_description'] = df.caption_alt_text_description
df = df.loc[df.caption_reference_description.notna()]
for col in tqdm(columns):
    df[col] = df[col].str.replace('\d+', '', regex=True)
    df[col] = df[col].str.lower()
    df[col] = df[col].str.capitalize()
    df[col] = df[col].apply(lambda x: str(x)+'.')

100%|██████████| 7/7 [00:00<00:00,  7.63it/s]


In [8]:
len(df.loc[df.caption_reference_description == df.caption_alt_text_description]) / len(df) * 100

0.010778184953653805

In [9]:
df = df.drop(columns=['caption_alt_text_description'])

In [10]:
df.to_csv('wiki_dataset/rus_wikitext_10k_train_normalized.csv')

# Тестим

In [11]:
import torch

In [12]:
!pip install torchmetrics==0.11.3

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [13]:
!pip install transformers==4.26.1

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [14]:
from torchmetrics.multimodal import CLIPScore

metric = CLIPScore(model_name_or_path="openai/clip-vit-base-patch16")

In [15]:
import torchvision.transforms as transforms
from PIL import Image
def read_image(path):
    img = Image.open(path).convert('RGB')

    resize = transforms.Resize([224, 224])
    img = resize(img)
    to_tensor = transforms.ToTensor()

    tensor = to_tensor(img)
    return tensor

In [16]:
images = torch.load('wiki_img_tensor.pt')
wikiScore = metric(images[0], df.caption_reference_description.loc[0])

In [17]:
wikiScore.detach()

tensor(19.9878)

In [19]:
scores_wiki = []
j = 0
for i in tqdm(df.index[:]):
    text = df.caption_reference_description.loc[i]
    try:
        #wikiScore = metric(read_image(f"wiki_dataset/images/image_{i}.jpg"), text)
        wikiScore = metric(images[j], text)
        scores_wiki.append(wikiScore.detach())
    except KeyboardInterrupt:
        print("stop")
        break
    except:
        # print(f"exception on {i}")
        pass
    j += 1
mean_wiki = sum(scores_wiki) / len(scores_wiki)

100%|██████████| 9278/9278 [28:53<00:00,  5.35it/s]


In [20]:
mean_wiki # оценка wiki

tensor(21.9187)

In [21]:
import torchvision.datasets as dset
coco_train = dset.CocoDetection(root = "coco_dataset/train2017",
                                annFile = "coco_dataset/annotations/captions_train2017.json")

loading annotations into memory...
Done (t=0.89s)
creating index...
index created!


In [27]:
scores_coco = []
for i in tqdm(range(9278)):
    image, captions = coco_train[i]
    name = str(captions[0]['image_id'])
    name = "0" * (12 - len(name)) + name
    name = f"coco_dataset/train2017/{name}.jpg"
    text = captions[0]['caption']
    cocoScore = metric(read_image(name), text)
    scores_coco.append(cocoScore.detach())
mean_coco = sum(scores_coco) / len(scores_coco)

100%|██████████| 9278/9278 [33:58<00:00,  4.55it/s]


In [28]:
mean_coco # оценка coco

tensor(30.4789)

In [30]:
!pip install googletrans==3.1.0a0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
/usr/share/python-wheels/urllib3-1.25.8-py2.py3-none-any.whl/urllib3/connectionpool.py:999: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pypi.ngc.nvidia.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
/usr/share/python-wheels/urllib3-1.25.8-py2.py3-none-any.whl/urllib3/connectionpool.py:999: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pypi.ngc.nvidia.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/

In [74]:
from googletrans import Translator

translator = Translator()
translated_text = []
for i in tqdm(range(5000)):
    image, captions = coco_train[i]
    text = captions[0]['caption']
    translated_text.append(translator.translate(text, dest="ru", src="en").text)

  1%|          | 29/5000 [00:10<29:24,  2.82it/s]


KeyboardInterrupt: 

In [68]:
scores_coco = []
for i in tqdm(range(1000)):
    image, captions = coco_train[i]
    name = str(captions[0]['image_id'])
    name = "0" * (12 - len(name)) + name
    name = f"coco_dataset/train2017/{name}.jpg"
    text = translated_text[i]
    try:
        cocoScore = metric(read_image(name), text)
        if cocoScore.detach() > 15:
            scores_coco.append(cocoScore.detach())
    except:
        #print(f"exception on {i}")
        pass
mean_coco = sum(scores_coco) / len(scores_coco)

100%|██████████| 1000/1000 [04:04<00:00,  4.09it/s]


In [69]:
len(scores_coco) / 1000 # процент норм переведенных описаний

0.913

In [67]:
mean_coco # оценка переведенного coco

tensor(18.9519)

# Перевод coco

In [19]:
n = 20000
coco_translation = []

In [25]:
len(coco_translation)

11218

In [27]:
from googletrans import Translator
import json

translator = Translator()
for i in tqdm(range(len(coco_translation), n)):
    image, captions = coco_train[i]
    name = str(captions[0]['image_id'])
    name = "0" * (12 - len(name)) + name
    name = f"coco_dataset/train2017/{name}.jpg"
    translated_captions = []
    for cur_cap in captions:
        text = translator.translate(cur_cap['caption'], dest="ru", src="en").text
        try:
            cocoScore = metric(read_image(name), text)
            if cocoScore.detach() > 15:
                translated_captions.append(text)
        except:
            #print(f"exception on {i}")
            pass
    coco_translation.append(translated_captions)
    if i % 1000 == 0:
        with open('coco_dataset/coco_captions_russian.json', 'w', encoding='utf-8') as f:
            json.dump(coco_translation, f, ensure_ascii=False, indent=4)


  0%|          | 29/8781 [04:47<24:07:34,  9.92s/it]

KeyboardInterrupt



In [24]:
with open('coco_dataset/coco_captions_russian.json', 'w', encoding='utf-8') as f:
    json.dump(coco_translation, f, ensure_ascii=False, indent=4)

In [74]:
# images = torch.load('wiki_img_tensor.pt')
# '''images = []
# exceptions = []
# captions = df.caption_reference_description
# for i in tqdm(df.index.tolist()):
#     try:
#         images.append(read_image("wiki_dataset/images/image_" + str(i) + ".jpg"))
#     except KeyboardInterrupt:
#         print("stop")
#         break
#     except:
#         exceptions.append(i)
#         print(f"exception on {i}")
# torch.save(images, 'wiki_img_tensor.pt')'''

In [21]:
# captions = [captions[i] for i in captions.index if i not in [18358, 19767, 24105]]
# len(captions)

9275